In [7]:
#!{sys.executable} -m pip install --upgrade pip
#Code to install packages you don't have already
#import sys
#!{sys.executable} -m pip install plotly

In [8]:
#Downloading all of the necessary packages
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import scipy as sp
import plotly.express as px
from pathlib import Path

In [ ]:
print("Data_Summary_Analysis is running.")

# Converts the text file into a csv file
df = pd.read_csv(file_path, encoding= 'unicode_escape')

#Adding a column name so that it is easier to call out. The column header had spaces.
df.columns=['Parse1']
#Removing the separation dashes
df= df[df.Parse1 !='--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------']


#Trying to clean up the file and the unecessary spaces using two steps
#STEP 1: Trimming space from the ends of each values
def trim_all_columns(df):
    #Trim whitespace from ends of each value across all series in dataframe
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)

#Utilize our trim definition for our dataframe
df = trim_all_columns(df)

#STEP2: Replacing every all greater that 1 space [' +'] with 1 space [' ']. Reason for this is so we can split the df by space
df = df['Parse1'].str.replace(' +',' ', regex = True)

In [ ]:
#Split the df by space. We are creating a total of 23 columns, therefore we use number 22 because (0,1,2,3,...,22)
df = df.str.split(' ', n = 22, expand=True)

#Adding proper column headers 
df.columns = ['Side','Grade','Source/Destination','All','Lpd','LpdN',
    'LpsED','uScr','Scr','Slip','Area (#)','Area (mm^2)','Avg','Median','STDV','Bin1','Bin2','Bin3',
    'Bin4','Bin5','Bin6','Bin7','Bin8']

In [ ]:
#Identify the index of the row we want to stop
#We want to stop the code when the 'WaferID' column reads 'wafer(s)'
row = df[df['Grade'] == 'wafer(s)'].index.tolist()[0]

#Keeping only the rows ip to the row we identfied
df = df.iloc[:row-3]

In [12]:
#Deleting irrelavent columns
del df['LpdN']
del df['LpsED']
del df['uScr']
del df['Scr']
del df['Slip']
del df['Area (#)']
del df['Area (mm^2)']
del df['Avg']
del df['Median']
del df['STDV']


In [13]:
# Removes extension from file name
fileName = Path(file_path).stem

# Extracts date and time from file name
fileTitle = ''.join(i for i in fileName if i.isdigit())

# Iterates through number string backward to isolate date and time
dateTime = ""
counter = 0
for character in fileTitle[::-1]:
    if counter <= 11:
        dateTime = "".join([dateTime, character])
        counter += 1
    else:
        break

dateTime = dateTime[::-1]

date = dateTime[0:8]

time = dateTime[8:]

# Extracts session from dataframe
a = df.iloc[0]['Grade']
b = df.iloc[0]['Source/Destination']
c = df.iloc[0]['All']

# Filter for session separated by a space
if b != None and c != None:
    session = " ".join([a, b, c])
elif b != None:
    session = " ".join([a, b])
else:
    session = a

# Extracts Lot ID from dataframe
a = df.iloc[1]['Source/Destination']

# Filter for lot ID separated by a space
b = df.iloc[1]['All']
if b == None:
    lotID = a
else:
    lotID = " ".join([a, b])

In [ ]:
# Adds additional columns to dataframe
df = df.assign(
    Session = lambda x: session,
    Lot_ID = lambda x: lotID,
    Date = lambda x: date,
    Time = lambda x: time
)

In [ ]:
# Removes irrelevant rows
df = df.drop(df[df.Side != 'F'].index).reset_index(drop=True)

In [ ]:
# Exports table as an excel file
df.to_csv(f'../../Wafer-Particles-main/Wafer-Particles-main/Parsed_Output/{fileName}.csv', encoding='utf-8', index=False)